In [2]:
import pandas as pd
import ast
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.metrics.pairwise import cosine_similarity
import re
from sklearn.decomposition import NMF
import numpy as np
import math

#### Förberedelse av datan

In [3]:
df_recipes = pd.read_csv('data/RAW_recipes.csv').dropna()
df_ratings = pd.read_csv('data/RAW_interactions.csv').dropna()

Se [EDA](eda.ipynb) för nogrannare förklaring av filtrering av datan

In [4]:
df_recipes = df_recipes[
    (df_recipes['minutes'] <= 1440) ##240 ### 1440
    & (df_recipes['minutes'] != 0)
    & (df_recipes['n_steps'] != 0)
    & (df_recipes['n_ingredients'] != 0)
]

changed = True
while changed:
    prev_len = len(df_ratings)
    
    # Filter users
    user_counts = df_ratings['user_id'].value_counts()
    valid_users = user_counts[user_counts >= 24].index ##22 ### 24
    df_ratings = df_ratings[df_ratings['user_id'].isin(valid_users)]
    
    # Filter recipes
    review_counts = df_ratings['recipe_id'].value_counts()
    valid_ids = review_counts[review_counts >= 18].index ##16 ### 16
    df_ratings = df_ratings[df_ratings['recipe_id'].isin(valid_ids)]
    df_recipes = df_recipes[df_recipes['id'].isin(valid_ids)]
    
    df_ratings = df_ratings[df_ratings['recipe_id'].isin(df_recipes['id'].values)]

    changed = len(df_ratings) != prev_len

df_ratings.reset_index(drop=True, inplace=True)
df_recipes.reset_index(drop=True, inplace=True)

'Minutes' (dvs. hur länge ett recept tar att laga) hade för det mesta tiderna som avrundat till närmsta 5 minuter, men vissa var ytterst specifica (t.ex 158 min). Delar upp tiderna i kategorier på 30 min intervaller så att små skillnader inte har en stor påverkan. Tar det 35 eller 36 min att laga är ju sak samma i verkligheten, men kommer skapa onöding 'noise' i datan.

In [5]:
bins = np.arange(0, 241, 30)  # 0,30,60,...,240
labels = [f"{i}-{i+30}" for i in bins[:-1]]
df_recipes['time_category'] = pd.cut(df_recipes['minutes'], bins=bins, 
                                     labels=labels, right=False, include_lowest=True)

In [6]:
df_recipes['tags'] = df_recipes['tags'].apply(lambda x: ast.literal_eval(x))
df_recipes['tags_processed'] = df_recipes['tags'].apply(lambda x: ' '.join(x))
df_recipes['ingredients'] = df_recipes['ingredients'].apply(lambda x: ast.literal_eval(x))

konverterar ingrediernserna till små bokstäver, tar bort mellanslag från individuella ingredienser ('olive oil' → 'oliveoil') så att de tokenize:s som en enhet. Raderar onödiga ord (t.ex. fresh) och insignifikanta ingredienser (t.ex salt, peppar) som inte bidrar mycket om receptet

In [ ]:
with open("garbage_words.txt", "r") as f:
        garbage_words = {line.strip().lower() for line in f if line.strip()}
        more = {'salt', 'black', 'water', 'sugar'} ## insignificant ingredients
        garbage_words = garbage_words.union(more)

def process_ingredients(lst):
    out = []
    for ing in lst:
        ing = ing.lower().strip()
        ing = re.sub(r'[^a-z0-9\- ]+', '', ing) ## removes everything except alpanumeric chars and '-'
        ing = re.sub(r'(?<!bell\s)pepper\b', '', ing) ## removes 'pepper' if not preceded by 'bell'
        ing = re.sub(r'\s+', ' ', ing) ## swaps any created space with single space
        words = [w for w in ing.split(' ') if w not in garbage_words]
        if words:
            out.append(''.join(words))
    return ' '.join(out) ## creates a string out of a list

In [8]:
df_recipes['ingredients_processed'] = df_recipes['ingredients'].apply(process_ingredients)

In [9]:
print(df_recipes.iloc[1234]['ingredients'])
print(df_recipes.iloc[1234]['ingredients_processed'])

['vegetable oil', 'all-purpose flour', 'onion', 'fresh parsley', 'celery', 'green bell pepper', 'red bell pepper', 'scallion', 'garlic', 'chicken broth', 'salt', 'creole seasoning', 'sausage']
vegetableoil all-purposeflour onion parsley celery greenbellpepper redbellpepper scallion garlic chickenbroth creoleseasoning sausage


In [10]:
with open("garbage_words.txt", "r") as f:
        garbage_words = {line.strip().lower() for line in f if line.strip()}
        
def clean_name(name:str):
        """Removes unnesseccary words from recipe names (like best, easy etc.) that cause noise."""
        pattern = r'\b(?:' + '|'.join(re.escape(w) for w in garbage_words) + r')\b'
        name = name.lower()
        name = re.sub(pattern, '', name)
        name = re.sub(r'\b\s+[sS]\s+\b', ' ', name) ## 's -> (space)s
        name = name.replace(r'\s+', ' ').strip() 
        return name

In [11]:
df_recipes['clean_name'] = df_recipes['name'].apply(clean_name)

In [12]:
print(df_recipes.iloc[1235]['name'])
print(df_recipes.iloc[1235]['clean_name'])

gwen s butter rich dinner rolls
gwen butter rich dinner rolls


#### Content recommender

Rekommenderar på basis av en `recipe seed`. Tar hänsyn till antal ingredienser, antal steg, tidskategori, taggar och själva ingredienserna.

`weights` är hur mycket de enskilda datapunkterna skall påverka slutliga resultatet för individuella recept.

**Numerisk data (n_steps, n_ingredients)** – MinMaxScaler
- Skalar värderna till mellan 0,1 då de har stor skala

**Tidskategori** – OneHotEncoder
-  Ger numeriska värden för kategorisk data så att de kan användas i beräkningar

**Ingredienser** – CountVectorizer
- Konverterar ingredienslistan till vektorer ("bag of words")
- Alla ingredienser ska ha samma vikt, oavsett hur många gånger de förekommer i datasettet, men ordning spelar inte roll

**Taggar** – TF-IDF
- Viktar ner ofta förekommande taggar 
- Många recept har samma generiska taggar som inte bidrar mycket

<br>

Räknar `cosine similarity` mellan de indivduella datapunkter och beräknar ett slutligt resultat genom att multiplicera varje värde med tillhörande vikt och addera dem ihop. Returnerar en `Series` med `recipe_id` som index och resultat som värde, som kan sedan användas i [HybridRecommender](#hybridrecommender) eller i `recommend_direct` för att få direkta rekommendationer.

<br>

**Vikter**

Ingredienser beskriver kanske bäst likheten hos recept och taggarna verkade informativa om receptets typ. Därför är de mest viktade.

Numeriska datan (n ingrediencer, n steg) kan beskriva komplexiteten hos ett recept, men beskriver inte innehållet.
Tid kan beskriva ansträngningen ett recept behöver, men är inte nödvändigtvis alltid så. Därför har de lägre vikt.


In [ ]:
class ContentRecommender:
    def __init__(self, df_recipes, weights={'tags':0.35, 'numeric_data':0.15, 'time': 0.15, 'ingredients':35}):
        """
        Args:
            df_recipes (Dataframe): the dataframe that contains the recipes.
            weights (dict): weights for calculating final score
        """
        self.garbage_words = garbage_words
        self.weights = weights
        #self.name_matrix = None
        self.numeric_matrix = None
        self.time_matrix = None
        self.ingredient_matrix = None
        self.tag_matrix = None

        self.recipe_indices = None
        self.recipes_df = df_recipes
    
    
    def fit(self):
        #name_tfidf = TfidfVectorizer(stop_words='english')
        scaler = MinMaxScaler()
        ohe = OneHotEncoder()
        countvec = CountVectorizer(stop_words='english')
        tag_tfidf = TfidfVectorizer(stop_words='english')

        #self.name_matrix = name_tfidf.fit_transform(self.recipes_df['clean_name'])
        self.numeric_matrix = scaler.fit_transform(self.recipes_df[['n_ingredients', 'n_steps']])
        self.time_matrix = ohe.fit_transform(self.recipes_df[['time_category']])
        self.ingredient_matrix = countvec.fit_transform(self.recipes_df['ingredients_processed'])
        self.tag_matrix = tag_tfidf.fit_transform(self.recipes_df['tags_processed'])

        self.recipe_indices = pd.Series(self.recipes_df.index, index=self.recipes_df['name'])


    def recommend(self, recipe_name):
        if recipe_name not in self.recipe_indices: return None
        idx = self.recipe_indices[recipe_name]
        
        #name_vector = self.name_matrix[idx]
        #name_simil = cosine_similarity(name_vector, self.name_matrix).flatten()

        num_data_vector = self.numeric_matrix[idx].reshape(1, -1)
        num_data_simil = cosine_similarity(num_data_vector, self.numeric_matrix).flatten()

        time_vector = self.time_matrix[idx]
        time_simil = cosine_similarity(time_vector, self.time_matrix).flatten()

        ingr_vector = self.ingredient_matrix[idx]
        ingr_simil = cosine_similarity(ingr_vector, self.ingredient_matrix).flatten()

        tag_vector = self.tag_matrix[idx]
        tag_simil = cosine_similarity(tag_vector, self.tag_matrix).flatten()

        
        scores =  num_data_simil * self.weights['numeric_data'] \
                + time_simil * self.weights['time'] \
                + ingr_simil * self.weights['ingredients'] \
                + tag_simil * self.weights['tags']
                #+ name_simil * self.weights['name']

        scores = [(i, score) for i, score in enumerate(scores) if i != idx]
        scores = pd.Series(
            data=[s for _, s in scores],
            index=self.recipes_df.iloc[[i for i, _ in scores]]['id']
        )
        scores.sort_values(ascending=False, inplace=True)
        return scores
    

    def recommend_direct(self, recipe_name, num_recs=10):
        """
        For calling the recommender directly.

        Args:
            recipe_name (str): The name of a recipe to use as the seed for recommendations
            num_recs (int): How many recommendations to produce

        Returns:
            list: The names of the recommended recipes
        """
        recs = self.recommend(recipe_name)
        if recs is None: return None
        return self.recipes_df[self.recipes_df['id'].isin(recs.index[:num_recs])]['name'].to_list()


In [482]:
contRec = ContentRecommender(df_recipes)
contRec.fit()

In [839]:
contRec.recommend_direct('land of nod  cinnamon buns', 10)

['apple enchiladas',
 'baked pork chops and apples',
 'bisquick coffee cake',
 'cinnamon flop',
 'fresh fruit fiesta bars',
 'overnight french toast casserole',
 'really really good candied sweet potatoes',
 'rhubarb crisp',
 'strawberry crisp for two',
 'sweet potato  yam  casserole with marshmallows']

>Rekommendationerna har ett sammanhang, men är ändå varierade. Det som står ut mest som den udda är *'baked pork chops and apples'*, men på vidare inspektion delar de en del ingredeienser och är på så sätt lika.

#### CollaborativeRecommender

Rekommenderar på basis av vad andra liknande användare har tyckt, tar `user Id` som input. Skapar en `user_item_matrix` med användare som rader, recept som kolumner och ratings som värden – fyller på med 0 var det saknas värde.

NMF bryter ner `user_item_matrix` till två matriser och söker mönster i datan. Räknar värdena med punktprodukten av användarens vektor och egenskaperna NMF har producerat. Raderar de recept som användaren har redan gett recension på (värdet är > 0)

Returnerar en `Series` med `recipe_id` som index och resultat som värde, som kan sedan användas i [HybridRecommender](#hybridrecommender) eller i `recommend_direct` för att få direkta rekommendationer.

In [ ]:
class CollaborativeRecommender():
    def __init__(self, df_recipes, df_ratings):
        """
        Args:
            df_recipes (DataFrame): the dataframe that contains the recipes.
            df_ratings (DataFrame): the dataframe that contains the ratings for recipes.
        """
        self.recipes_df = df_recipes
        self.ratings_df = df_ratings
        
        self.nmf_model = None
        self.user_item_matrix = None
        self.user_mapper = None
        self.recipe_mapper = None

    def fit(self):

        self.user_item_matrix = self.ratings_df.pivot(columns='recipe_id', index='user_id', values='rating').fillna(0)
        self.user_mapper = {user_id: i for i, user_id in enumerate(self.user_item_matrix.index)}
        self.recipe_mapper = {recipe_id: i for i, recipe_id in enumerate(self.user_item_matrix.columns)}

        nmf = NMF(n_components=30, init='random', random_state=42, max_iter=600) ## 30
        self.nmf_model = nmf.fit(self.user_item_matrix)

    def recommend(self, user_id):
        #self.fit()       
        if user_id not in self.user_mapper: return None

        user_idx = self.user_mapper[user_id]
        user_vector = self.user_item_matrix.iloc[user_idx].values
        user_p = self.nmf_model.transform(user_vector.reshape(1, -1))
        pred_ratings = np.dot(user_p, self.nmf_model.components_)

        preds = pd.Series(data=pred_ratings[0], index=self.recipe_mapper.keys())
        preds = preds[~(user_vector > 0)].sort_values(ascending=False)
        return preds
        
    
    def recommend_direct(self, user_id, num_recs=10):
        """
        For calling the recommender directly.

        Args:
            user_id (int): The id of the user to produce recommendations for.
            num_recs (int): How many recommendations to produce.

        Returns:
            list: The names of the recommended recipes.
        """
        recs = self.recommend(user_id)
        if recs is None: return None
        return self.recipes_df[self.recipes_df['id'].isin(recs.index[:num_recs])]['name'].to_list()

In [842]:
colRec = CollaborativeRecommender(df_recipes, df_ratings)
colRec.fit()

In [843]:
colRec.recommend_direct(user_id=139930)

['whatever floats your boat  brownies',
 'banana banana bread',
 'best ever banana cake with cream cheese frosting',
 'creamy burrito casserole',
 'crock pot chicken with black beans   cream cheese',
 'crumb topped banana muffins',
 'delicious chicken pot pie',
 'denny s style french toast',
 'kittencal s famous greek salad',
 'pulled pork  crock pot']

In [860]:
ids = df_ratings[df_ratings['user_id'] == 139930]['recipe_id']
df_recipes[df_recipes['id'].isin(ids.values)]['name']

114                        asian steamed dumpling filling
127                        authentic italian tomato sauce
489                                        cherry cookies
803                                     crock pot special
805                                     crock pot stifado
959     easy peezy pizza dough  bread machine pizza dough
1101    fudge crinkles  a great 4 ingredient cake mix ...
1102                                    fudge filled bars
1192                               greek style oven fries
1374                inside out stuffed green bell peppers
1397                    italian pepper and sausage dinner
1451                       kelly s southwestern beef stew
1464       kittencal s 5 minute cinnamon flop brunch cake
1465    kittencal s baked potato salad casserole  or c...
1469    kittencal s banana cinnamon snack cake or muff...
1484                 kittencal s chocolate frosting icing
1495                          kittencal s famous coleslaw
1505     kitte

>På basis av vad denna användare tyckt, tycker jag att rekommendationerna verkar rimliga och möjligen som något användaren kunde gilla.

#### HybridRecommender

Söker rekommendationerna från `ContentRecommender` och `CollaborativeRecommender`, normaliserar värdena på rekommendationerna så de är i samma skala. Räknar sedan de nya värdena med vikterna, sorterar och returnerar top `num_recs` rekommendationer. 

I fall `user_id` är ogiltigt returnerar den endast innehållsrekommendationer, och på samma sätt, om inte `recipe_seed` inte är giltigt returnerar den endast kollaborativa rekommendationer.

Returnerar en `DataFrame` med recept id som index och namn som kolumn. Detta för att kunna använda recept id för att räkna i [Evaluator](#evaluator) då det fanns skilda recept med samma namn.

In [ ]:
class HybridRecommender():
    def __init__(self, recipes_df, ratings_df, weights={'collaborative': 0.7, 'content': 0.3}):
        """
        Args:
            recipes_df (DataFrame): the dataframe that contains the recipes.
            ratings_df (DataFrame): the dataframe that contains the ratings for the recipes.
            weights (Dict): weights for calculating final score.
        """
        self.recipes_df = recipes_df
        self.ratings_df = ratings_df
        self.weights = weights

        self.contRec = ContentRecommender(recipes_df)
        self.collRec = CollaborativeRecommender(recipes_df, ratings_df)

    def fit(self):
        self.contRec.fit()
        self.collRec.fit()
        
    def recommend(self, recipe_seed, user_id, num_recs=10):
        """
        For generating recommendations.

        Args:
            recipe_seed (str): The name of a recipe to use as the seed for recommendations
            user_id (int): The id of the user to produce recommendations for.
            num_recs (int): How many recommendations to produce.

        Returns:
            DataFrame: Id, Name of the recommended recipes.
        """
        cont_recs = self.contRec.recommend(recipe_name=recipe_seed)
        coll_recs = self.collRec.recommend(user_id=user_id)

        if cont_recs is None and coll_recs is None:
            print('Invalid input')
            return None

        ## both return None if input is invalid
        if cont_recs is None: return self.recipes_df[self.recipes_df['id'].isin(coll_recs.index[:num_recs])]['name'].to_list()
        if coll_recs is None: return self.recipes_df[self.recipes_df['id'].isin(cont_recs.index[:num_recs])]['name'].to_list()
        
        ## Normalise scores so one doesn't weigh dispropotinally (coll_recs returend values > 1)
        cont_recs = (cont_recs - cont_recs.min()) / (cont_recs.max() - cont_recs.min())
        coll_recs = (coll_recs - coll_recs.min()) / (coll_recs.max() - coll_recs.min())

        ## Align so both have all id's, fill empty with 0 (avoids NaN on weighing)
        cont_recs, coll_recs = cont_recs.align(coll_recs, join='outer', fill_value=0)

        hybrid_recs = cont_recs * self.weights['collaborative'] + coll_recs * self.weights['content']
        recs = hybrid_recs.sort_values(ascending=False).head(num_recs)
        
        res = self.recipes_df[['id', 'name']].set_index('id').loc[recs.index]

        return res



In [846]:
hybrid = HybridRecommender(df_recipes, df_ratings)
hybrid.fit()

In [847]:
hybrid.recommend('land of nod  cinnamon buns', 139930)

,name
43509,crumb topped banana muffins
21859,apple enchiladas
122412,strawberry crisp for two
41541,really really good candied sweet potatoes
90771,baked pork chops and apples
25885,banana banana bread
82925,edna s apple crumble aka apple crisp
32554,cinnamon flop
93946,rhubarb crisp
104150,sweet potato yam casserole with marshmallows


Resultatet är en blandning av innehållsbaserade och kollaborativa rekommendationer. Rekommendationerna är i helt ny ordning, då de fått nya värden av viktandet.

För denna person med detta recept, består majoriteten av rekommendationerna utav de innehållsbaserade, top rekommendationen är dock ifrån de kollaborativa. Det har också kommit en totalt ny rekommendation, som antagligen har varit just utanför top 10 i någon dera rekommendation och nu med kombinationen av båda blivit förstärkt och klättrat upp. 

In [855]:
df_ratings[df_ratings['user_id'] == 139930]['rating'].value_counts()

rating
5    22
4     4
3     2
Name: count, dtype: int64

Användaren har gett 28 st ratings, och med tanke på att datan innehåller tusentals recept är det inte konstigt att innehållsbaserade rekommendationerna i det här fallet får mera tyngd i slutresultatet, då det kan vara svårare att dra slutsatser av denna användares preferencer då det i förhållande till datasettet finns väldigt lite data om denna användare och antagligen väldigt lite överlappning med andra användare.

#### Evaluator

In [867]:
class Evaluator():
    def __init__(self, df_recipes, df_ratings, recipe_seed='to die for crock pot roast'):
        self.recommender = HybridRecommender(df_recipes, df_ratings)
        self.recipes_df = df_recipes
        self.ratings_df = df_ratings
        self.popularity_scores = None
        self.name_to_id = None
        self.recipe_seed = recipe_seed
        

    def fit(self):
        self.recommender.fit()

        rating_counts = self.ratings_df['recipe_id'].value_counts()
        num_users = self.ratings_df['user_id'].nunique()
        self.popularity_scores = rating_counts / num_users

        self.name_to_id = pd.Series(self.recipes_df['id'].values, index=self.recipes_df['name'])

    def generate_all_recommendations(self):
        all_recs = {}
        
        for user_id in self.ratings_df['user_id'].unique():
            recs = self.recommender.recommend(self.recipe_seed, user_id, 10)
            all_recs[user_id] = recs
        return all_recs
    
    def calculate_precision_at_k(self, all_recommendations, k=10):
        precision = []
        for uid, recs in all_recommendations.items():
            liked = self.ratings_df[(self.ratings_df['user_id'] == uid) & (self.ratings_df['rating'] >= 4)]
            #rec_ids = [self.name_to_id[r] for r in recs]
            hits = len(liked[liked['recipe_id'].isin(recs.index)])
            precision.append(hits/k)
        
        return sum(precision) / len(precision)
    
    def calculate_coverage(self, all_recommendations):

        unique_names = {v for val in all_recommendations.values() for v in val}
        return len(unique_names) / self.recipes_df['name'].nunique()
    
    def calculate_novelty(self, all_recommendations):
     
        novelty = []
        for uid, recs in all_recommendations.items():
            scores = [-math.log2(self.popularity_scores[idx]) for idx in recs.index]
            novelty.append(sum(scores) / len(scores))
        
        return sum(novelty) / len(novelty)

In [868]:
evaluator = Evaluator(df_recipes, df_ratings)
evaluator.fit()

In [869]:
all_recs = evaluator.generate_all_recommendations()
    
precision = evaluator.calculate_precision_at_k(all_recs)
coverage = evaluator.calculate_coverage(all_recs)
novelty = evaluator.calculate_novelty(all_recs)

In [870]:
print("\n--- Evaluation Metrics ---")
print(f"Average Precision@10: {precision:.4f}")
print(f"Catalog Coverage: {coverage:.4f}")
print(f"Average Novelty: {novelty:.4f}")


--- Evaluation Metrics ---
Average Precision@10: 0.0124
Catalog Coverage: 0.0004
Average Novelty: 6.0060


#### Resultat och analys

Kunde ha justerat vikter och spelat runt med datan i all oändlighet för att försöka slipa på resultatet, men i allmänhet hade filtrering av datan, lite beroende på raderar man fler recept eller användare, tendens att öka precision och coverage, men sänka novelty en aning (några tion-/hundradelar). Aggressiv filtrering ökar densiteten av datan (mindre nollor i `user_item_matrix`), men samtidigt tappar man recept som inte har lika många ratings och datan börjar luta sig mot endast populära recept. Dock är datan även med denna filtering fortfarande ganska gläs.

Högre vikt på `collaborative` rekommendationer i `hybridRecommender` hade liknande effekt som filtrering av datan – precision och coverage ökar, novelty sjunker en aning. Detta kan också bero på att om rekommendationerna från kollaborativa systemet har allmänt lägre beräknat värde (pga. liten överlappning och då svårighet att ge meningsfulla rekommendationer) behöver de viktas högre för att inte bli utfiltrerade av innehållsrekommendationerna.

Att inte använda receptnamn i `contentRecommender` producerade även bättre resultat i evaluering. Detta verkar helt rimligt då flera receptnamn är 'noisy' var de är skämtsamma etc., och inte nödvändigtvis berättar så mycket om själva receptet. 

Lite beroende på filtreringen av datan gav 30-40 `n_components` bäst resultat. Initsialisering som 'random' eller 'nndsv' producerade bästa, lika, precision och coverage.

Presision och coverage är inte fantastiska: 1.24% av rekommendationerna är relevanta och endast 0.4% av all data används i rekommendationerna. Novelty är medel, rekommendationerna består av både populära recept och också mer nichade, oväntade, rekommendationer.

Som slutsats: datan behövde filtreras rejält för att få någorlunda resultat. Kvaliten på metadatan om recepten var sisådär (då det är producerat av community) och för att få bättre resultat behövde systemet vikta mera på kollaborativa rekommendationer.